## Instructions

- Follow the installation instructions in the readme file
- Answer the questions in this notebook
- Once your work is finished: restart the kernel, run all cells in order and check that the outputs are correct.
- Send your completed notebook to `remy.degenne@inria.fr` with email title `SL_TP4_NAME1_NAME2` (or `SL_TP4_NAME` if you work alone).

**Deadline: January 23, 15:00 CET**.

In [72]:
# This cell is setting up google colab. Ignore it if you work locally.
if 'google.colab' in str(get_ipython()):
    print("Installing packages, please wait a few moments. Restart the runtime after the installation.")
    # install rlberry library
    !pip install scipy scikit_learn git+https://github.com/rlberry-py/rlberry

Installing packages, please wait a few moments. Restart the runtime after the installation.
  Cloning https://github.com/rlberry-py/rlberry to /tmp/pip-req-build-ynq78h7z
  Running command git clone --filter=blob:none --quiet https://github.com/rlberry-py/rlberry /tmp/pip-req-build-ynq78h7z
  Resolved https://github.com/rlberry-py/rlberry to commit ad3febee482cd7fa5ae15b3e37081d6d6451ee9e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


# Fitted Q Iteration (FQI)

## Introduction

In this notebook, you will implement the Fitted Q Iteration algorithm (FQI) to solve the [CartPole](https://gymnasium.farama.org/environments/classic_control/cart_pole/) problem.

This notebooks will first cover the basics for using the Gymnasium library: how to instantiate an environment, step into it and collect training data from the FQI algorithm.

You will then learn how to implement step-by-step the FQI algorithm which is the predecessor of the [Deep Q-Network (DQN)](https://stable-baselines3.readthedocs.io/en/master/modules/dqn.html) algorithm.

In [73]:
import numpy as np
import gymnasium as gym
from gymnasium import spaces
from dataclasses import dataclass

import os

from functools import partial
from pathlib import Path
from typing import Optional

from sklearn import tree
from sklearn.base import RegressorMixin
from sklearn.exceptions import NotFittedError
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neighbors import KNeighborsRegressor

## First steps with the Gym interface

An environment that follows the [gym interface](https://gymnasium.farama.org/) is quite simple to use.
It provides to this user mainly three methods, which have the following signature (for gym versions > 0.26):

- `reset()` called at the beginning of an episode, it returns an observation and a dictionary with additional info (defaults to an empty dict)
- `step(action)` called to take an action with the environment, it returns the next observation, the immediate reward, whether new state is a terminal state (episode is finished), whether the max number of timesteps is reached (episode is artificially finished), and additional information
- (Optional) `render()` which allow to visualize the agent in action. Note that graphical interface does not work on google colab, so we cannot use it directly (we have to rely on `render_mode='rbg_array'` to retrieve an image of the scene).

Under the hood, it also contains two useful properties:
- `observation_space` which is one of the gym spaces (`Discrete`, `Box`, ...) and describe the type and shape of the observation
- `action_space` which is also a gym space object that describes the action space, so the type of action that can be taken

The best way to learn about [gym spaces](https://gymnasium.farama.org/api/spaces/) is to look at the [source code](https://github.com/Farama-Foundation/Gymnasium/tree/main/gymnasium/spaces), but you need to know at least the main ones:
- `gym.spaces.Box`: A (possibly unbounded) box in $R^n$. Specifically, a Box represents the Cartesian product of n closed intervals. Each interval has the form of one of $[a, b]$, $(-\infty, b]$, $[a, \infty)$, or $(-\infty, \infty)$. Example: A 1D-Vector or an image observation can be described with the Box space.
```python
# Example for using image as input:
observation_space = spaces.Box(low=0, high=255, shape=(HEIGHT, WIDTH, N_CHANNELS), dtype=np.uint8)
```                                       

- `gym.spaces.Discrete`: A discrete space in $\{ 0, 1, \dots, n-1 \}$
  Example: if you have two actions ("left" and "right") you can represent your action space using `Discrete(2)`, the first action will be 0 and the second 1.

## CartPole Environment

For this example, we will use CartPole environment, a classic control problem.

"A pole is attached by an un-actuated joint to a cart, which moves along a frictionless track. The system is controlled by applying a force of +1 or -1 to the cart. The pendulum starts upright, and the goal is to prevent it from falling over. A reward of +1 is provided for every timestep that the pole remains upright. "

Cartpole environment: [https://gymnasium.farama.org/environments/classic_control/cart_pole/](https://gymnasium.farama.org/environments/classic_control/cart_pole/)

![Cartpole](https://cdn-images-1.medium.com/max/1143/1*h4WTQNVIsvMXJTCpXm_TAw.gif)

In [74]:
# Instantiate the environment
env = gym.make("CartPole-v1")

In [75]:
# Box(4,) means that it is a Vector with 4 components
print("Observation space:", env.observation_space)
print("Shape:", env.observation_space.shape)

# Discrete(2) means that there is two discrete actions
print("Action space:", env.action_space)

Observation space: Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)
Shape: (4,)
Action space: Discrete(2)


In [76]:
# The reset method is called at the beginning of an episode
obs, info = env.reset()

In [77]:
# Sample a random action
action = env.action_space.sample()
print(f"Sampled action: {action}")

Sampled action: 0


In [78]:
# step in the environment
obs, reward, terminated, truncated, info = env.step(action)

In [79]:
# Note the obs is a numpy array
# info is an empty dict for now but can contain any debugging info
# reward is a scalar
print(obs.shape, reward, terminated, truncated, info)

(4,) 1.0 False False {}


### Exercise: write the function to collect data

This function collects a dataset of transitions that will be used to train a model using the FQI algorithm.

See docstring of the function for what is expected as input/output.

In [80]:
@dataclass
class OfflineData:
    """
    A class to store transitions.
    """

    observations: np.ndarray  # same as "state" in the theory
    next_observations: np.ndarray
    actions: np.ndarray
    rewards: np.ndarray
    terminateds: np.ndarray

In [104]:
def collect_data(env: gym.Env, n_steps: int = 50_000) -> OfflineData:
    """
    Collect transitions using a random agent (sample action randomly).

    :param env: The environment.
    :param n_steps: Number of steps to perform in the env.
    :return: The collected transitions.
    """

    assert isinstance(env.observation_space, spaces.Box)
    # Numpy arrays (buffers) to collect the data
    observations = np.zeros((n_steps, *env.observation_space.shape))
    next_observations = np.zeros((n_steps, *env.observation_space.shape))
    # Discrete actions
    actions = np.zeros((n_steps, 1))
    rewards = np.zeros((n_steps,))
    terminateds = np.zeros((n_steps,))

    # Variable to know if the episode is over (done = terminated or truncated)
    done = False
    # Start the first episode
    obs, _ = env.reset()

    ### YOUR CODE HERE
    # You need to collect transitions for `n_steps` using
    # a random agent (sample action uniformly).
    # Do not forget to reset the environment if the current episode is over
    # (done = terminated or truncated)
    # 1. Sample a random action
    # 2. Step in the env using this random action
    # 3. Retrieve the new transition data (observation, reward, ...)
    #  and update the numpy arrays (buffers)
    # 4. Repeat until you collected `n_steps` transitions
    for i in range(n_steps):
        action = env.action_space.sample() # we sample a random action
        next_obs, reward, terminated, truncated, _ = env.step(action) # we step in the env using this random action

        #we update the numpy arrays
        observations[i] = obs 
        next_observations[i] = next_obs 
        actions[i] = action 
        rewards[i] = reward 
        terminateds[i] = terminated or truncated # we update the variable done to know if the episode is over

        obs = next_obs

        if terminated or truncated: # if the episode is over, we reset the environment
            obs, _ = env.reset()

    ### END OF YOUR CODE

    return OfflineData(
        observations,
        next_observations,
        actions,
        rewards,
        terminateds,
    )

Let's try the collect data method:

In [105]:
env_id = "CartPole-v1"
env = gym.make(env_id)
n_steps = 10_000
# Collect transitions for n_steps
data = collect_data(env=env, n_steps=n_steps)
# Check the length of the collected data
assert len(data.observations) == n_steps
assert len(data.actions) == n_steps
# Check that there are multiple episodes in the data
assert not np.all(data.terminateds)
assert np.any(data.terminateds)
# Check the shape of the collected data
if env_id == "CartPole-v1":
    assert data.observations.shape == (n_steps, 4)
    assert data.next_observations.shape == (n_steps, 4)
assert data.actions.shape == (n_steps, 1)
assert data.rewards.shape == (n_steps,)

## Fitted Q Iteration (FQI) Agent

See Lecture 4, slide 31 (and next slides for more explanations in the linear case, although this practical session is not linear).

At each iteration of the algorithm, a dataset of transitions is gathered. Then target Q values for each transition are computed and the algorithm solves a regression problem with the transitions as inputs and the target values as outputs to update its Q-value approximation.

After the maximal number of iterations is reached, the policy returned is the greedy policy with respect to the current Q-values.

### Choosing a regression model

With FQI, you can use any regression model to produce a Q-value estimator from a dataset of transitions and targets.

Here we are choosing a [k-nearest neighbors regressor](https://scikit-learn.org/stable/modules/neighbors.html#regression), but one could choose a linear model, a decision tree, a neural network, ...

In [106]:
# First choose the regressor
model_class = partial(
    KNeighborsRegressor, n_neighbors=30
)  # LinearRegression, GradientBoostingRegressor...

### 1. Exercise: write the function to predict Q-Values

In FQI, we will need to compute, for any transition $(s, a, r, s')$, a target value $y = r + \gamma \cdot \max_{a' \in A}(Q^{n-1}_\theta(s', a'))$. In order to do that, we need to be able to compute the current Q-value of state-action pairs.

See docstring of the function for what is expected as input/output.

In [107]:
def get_q_values(
    model: RegressorMixin,
    obs: np.ndarray,
    n_actions: int,
) -> np.ndarray:
    """
    Retrieve the q-values for a set of observations obs.
    Q(s, action) for all s in obs and all possible actions.

    :param model: Q-value estimator
    :param obs: A batch of observations
    :param n_actions: Number of discrete actions.
    :return: The predicted q-values for the given observations
        (batch_size, n_actions)
    """
    batch_size = len(obs)
    q_values = np.zeros((batch_size, n_actions))

    ### YOUR CODE HERE
    # for every possible actions a:
    # 1. Create the regression model input $(s, a)$ for the action a
    # and states s (here a batch of observations)
    # 2. Predict the q-values for the batch of states (use model.predict)
    # 3. Update q-values array for the current action a
    for a in range(n_actions):
        model_input = np.concatenate((obs, np.ones((batch_size, 1)) * a), axis=1) # we create the regression model input
        q_values[:, a] = model.predict(model_input) # we predict the q-values for the batch of states and update q-values array for the current action a



    ### END OF YOUR CODE

    return q_values

### Create the Agent
To create an agent, rlberry requires to use a **very simple interface**, with basically two methods to implement: `fit()` and `eval()`.

You can find more information on this interface [here(AgentWithSimplePolicy)](rlberry.agents.agent.AgentWithSimplePolicy).
### function fit() :

#### 1. First Iteration

For $n = 0$, the initial training set is defined as:

- $x = (s_t, a_t)$
- $y = r_t$

We fit a regression model $f_\theta(x) = y$ to obtain $ Q^{n=0}_\theta(s, a) $



#### 2. Exercise: the fitted Q iterations

1. Create the training set based on the previous iteration $ Q^{n-1}_\theta(s, a) $ and the transitions:
- input: $x = (s_t, a_t)$
- if $s_{t+1}$ is non-terminal: $y = r_t + \gamma \cdot \max_{a' \in A}(Q^{n-1}_\theta(s_{t+1}, a'))$
- if $s_{t+1}$ is terminal, do not bootstrap: $y = r_t$

2. Fit a model $f_\theta$ using a regression algorithm to obtain $ Q^{n}_\theta(s, a)$

\begin{aligned}
 f_\theta(x) = y
\end{aligned}

4. Repeat, $n = n + 1$

### function evaluate() :

#### 3. Exercise: write the function to evaluate a model

A greedy policy $\pi(s)$ can be defined using the q-value:

$\pi(s) = argmax_{a \in A} Q(s, a)$.

It is the policy that take the action with the highest q-value for a given state.





In [108]:
from rlberry.agents import Agent
from gymnasium.wrappers.monitoring.video_recorder import VideoRecorder


class Fitted_Q_Iteration(Agent):
    name = "Fitted_Q_Iteration"

    def __init__(
        self,
        env,
        model_class,
        n_steps_collection=50_000,  # Number of steps to perform in the env to collect data.
        eval_freq=2,
        n_eval_episodes=10,
        gamma=0.99,
        **kwargs,
    ):
        # it's important to put **kwargs to ensure compatibility with the base class
        # self.env is initialized here
        super().__init__(env=env, **kwargs)

        self.model_class = model_class  # The Agent's regression model class
        self.eval_freq = eval_freq  # How often do we evaluate the learned model
        self.n_eval_episodes = n_eval_episodes  # How many episodes to evaluate every eval-freq

        self.gamma = 0.99  # discount factor
        self.n_actions = int(self.env.action_space.n)  # Number of discrete actions

        # Collect data : transitions for n_steps
        self.current_data = collect_data(env=self.env, n_steps=n_steps_collection)

        # 1 - First iteration:
        # The target q-value is the reward obtained
        targets = self.current_data.rewards.copy()
        # Create input for current observations and actions
        # Concatenate the observations and actions
        # so we can predict qf(s_t, a_t)
        self.current_obs_input = np.concatenate(
            (self.current_data.observations, self.current_data.actions), axis=1
        )
        # Fit the estimator for the current target
        self.current_model = model_class().fit(self.current_obs_input, targets)

    def fit(self, budget, **kwargs):
        # 2 - the fitted Q iterations
        for iter_idx in range(budget):
            ### YOUR CODE HERE
            # TODO:
            # 1. Compute the q values for the next states using
            # the previous regression model
            # 2. Keep only the next q values that correspond
            # to the greedy-policy
            # 3. Construct the regression target (TD(0) target)
            # 4. Fit a new regression model with this new target

            next_q_values = get_q_values(self.current_model, self.current_data.next_observations, self.n_actions)

            # Select max Q-value for each next observation
            max_next_q_values = np.max(next_q_values, axis=1)

            # Construct TD(0) target
            targets = self.current_data.rewards + self.gamma * max_next_q_values * (1 - self.current_data.terminateds)

            # Fit new regression model
            self.current_model = self.model_class().fit(self.current_obs_input, targets)


            ### END OF YOUR CODE

            if (iter_idx + 1) % self.eval_freq == 0:
                print(f"Iter {iter_idx + 1}")
                print(
                    f"Score: {self.current_model.score(self.current_obs_input, targets):.2f}"
                )
                final_eval_result = self.eval(self.n_eval_episodes)

        info = {"final_eval_result": final_eval_result}
        return info  # return the final eval mean, but more informations are directly displayed by the eval() function in the output terminal.

    # 3 - function to evaluate a model
    def eval(
        self,
        n_simulations: int = 10,
        video_name: Optional[str] = None,
    ) -> None:
        episode_returns, episode_reward = [], 0.0
        total_episodes = 0
        done = False

        if not self.eval_env:
            self.eval_env = self.env

        # Setup video recorder
        video_recorder = None
        if video_name is not None and self.eval_env.render_mode == "rgb_array":
            os.makedirs("./logs/videos/", exist_ok=True)

            video_recorder = VideoRecorder(
                env=self.eval_env,
                base_path=f"./logs/videos/{video_name}",
            )

        obs, _ = self.eval_env.reset()
        n_actions = int(self.eval_env.action_space.n)
        assert isinstance(
            self.eval_env.action_space, spaces.Discrete
        ), "FQI only support discrete actions"

        while total_episodes < n_simulations:
            # Record video
            if video_recorder is not None:
                video_recorder.capture_frame()

            ### YOUR CODE HERE

            # Retrieve the q-values for the current observation
            # you need to re-use `get_q_values()`
            # Then select the action that maximizes the q-value for each state
            # Do a step in the env using the selected action

            # Retrieve the q-values for the current observation
            q_values = get_q_values(self.current_model, obs.reshape(1, -1), n_actions)

            # Select the action that maximizes the q-value for the current state
            action = np.argmax(q_values)

            # Do a step in the environment using the selected action
            #next_obs, reward, terminated, truncated, _ = self.eval_env.step(action)
            obs, reward, terminated, truncated, _ = self.eval_env.step(action)


            ### END OF YOUR CODE

            episode_reward += float(reward)

            done = terminated or truncated
            if done:
                episode_returns.append(episode_reward)
                episode_reward = 0.0
                total_episodes += 1
                obs, _ = self.eval_env.reset()
            #else :
             #   obs = next_obs  # Update the observation


        if video_recorder is not None:
            print(f"Saving video to {video_recorder.path}")
            video_recorder.close()

        print(
            f"Total reward = {np.mean(episode_returns):.2f} +/- {np.std(episode_returns):.2f}"
        )

        return np.mean(episode_returns)

### Performing experiments

First, let's define some constants:

In [109]:
from rlberry.envs import gym_make
from rlberry.manager import ExperimentManager

# Max number of iterations
n_iterations = 20
# How often do we evaluate the learned model
eval_freq = 2
# How many episodes to evaluate every eval-freq
n_eval_episodes = 10
# discount factor
gamma = 0.99
# Number of discrete actions
n_actions = int(env.action_space.n)


env_id = "CartPole-v1"  # Id of the environment
env_ctor = gym_make  # constructor for the env
env_kwargs = dict(id=env_id)  # give the id of the env inside the kwargs

eval_env_kwargs = dict(id=env_id, render_mode="rgb_array")

Now let's create an `ExperimentManager`, which is a class used to run experiments with specified agents and environments.

The experiment manager spawns agents and environments for training and then once the agents are trained, it uses these agents and new environments to evaluate how well the agents perform. All of these steps can be done several times to assess stochasticity of agents and/or environment.

In [110]:
my_experiment = ExperimentManager(
    Fitted_Q_Iteration,  # Agent Class
    (env_ctor, env_kwargs),  # Environment as Tuple(constructor,kwargs)
    init_kwargs=dict(
        model_class=model_class,
        n_steps_collection=50_000,
        eval_freq=eval_freq,
        n_eval_episodes=n_eval_episodes,
        gamma=gamma,
    ),
    eval_env=(env_ctor, eval_env_kwargs),
    fit_budget=int(n_iterations),  # Budget used to call our agent "fit()"
    n_fit=1,  # Number of agent instances to fit.
    agent_name="Agent_FQI_" + env_id,  # Name of the agent
    seed=42,
)

Use `fit()` to train an agent and then `eval_agents()` to evaluate the trained agents.

In [111]:
my_experiment.fit()

[INFO] 10:59: Running ExperimentManager fit() for Agent_FQI_CartPole-v1 with n_fit = 1 and max_workers = None. 
INFO:rlberry_logger:Running ExperimentManager fit() for Agent_FQI_CartPole-v1 with n_fit = 1 and max_workers = None.


Iter 2
Score: 0.73
Total reward = 22.20 +/- 9.89
Iter 4
Score: 0.86
Total reward = 135.90 +/- 9.45
Iter 6
Score: 0.90
Total reward = 157.80 +/- 27.95
Iter 8
Score: 0.92
Total reward = 177.70 +/- 30.62
Iter 10
Score: 0.93
Total reward = 173.60 +/- 27.33
Iter 12
Score: 0.93
Total reward = 190.70 +/- 48.67
Iter 14
Score: 0.94
Total reward = 337.20 +/- 58.65
Iter 16
Score: 0.94
Total reward = 419.00 +/- 85.17
Iter 18
Score: 0.94
Total reward = 361.90 +/- 100.73
Iter 20
Score: 0.94
Total reward = 400.10 +/- 88.26


[INFO] 11:00: ... trained! 
INFO:rlberry_logger:... trained!
[INFO] 11:00: Saved ExperimentManager(Agent_FQI_CartPole-v1) using pickle. 
INFO:rlberry_logger:Saved ExperimentManager(Agent_FQI_CartPole-v1) using pickle.
[INFO] 11:00: The ExperimentManager was saved in : 'rlberry_data/temp/manager_data/Agent_FQI_CartPole-v1_2024-01-12_10-59-20_0be04f64/manager_obj.pickle' 
INFO:rlberry_logger:The ExperimentManager was saved in : 'rlberry_data/temp/manager_data/Agent_FQI_CartPole-v1_2024-01-12_10-59-20_0be04f64/manager_obj.pickle'


### Record a video of the trained agent

In [112]:
video_name = f"FQI_{env_id}"
n_eval_episodes = 3
my_experiment.eval_agents(
    eval_kwargs=dict(n_simulations=n_eval_episodes, video_name=video_name)
)

[INFO] Evaluation:INFO:rlberry_logger:[INFO] Evaluation:


Saving video to ./logs/videos/FQI_CartPole-v1.mp4
Moviepy - Building video ./logs/videos/FQI_CartPole-v1.mp4.
Moviepy - Writing video ./logs/videos/FQI_CartPole-v1.mp4



.INFO:rlberry_logger:.


Moviepy - Done !
Moviepy - video ready ./logs/videos/FQI_CartPole-v1.mp4
Total reward = 382.00 +/- 0.00
Saving video to ./logs/videos/FQI_CartPole-v1.mp4
Moviepy - Building video ./logs/videos/FQI_CartPole-v1.mp4.
Moviepy - Writing video ./logs/videos/FQI_CartPole-v1.mp4



.INFO:rlberry_logger:.
  Evaluation finished 
INFO:rlberry_logger:  Evaluation finished 



Moviepy - Done !
Moviepy - video ready ./logs/videos/FQI_CartPole-v1.mp4
Total reward = 500.00 +/- 0.00


[382.0, 500.0]

In [113]:
import base64
from IPython import display as ipythondisplay


def show_videos(video_path: str = "", prefix: str = "") -> None:
    """
    Taken from https://github.com/eleurent/highway-env

    :param video_path: Path to the folder containing videos
    :param prefix: Filter the video, showing only the only starting with this prefix
    """
    html = []
    for mp4 in Path(video_path).glob(f"{prefix}*.mp4"):
        video_b64 = base64.b64encode(mp4.read_bytes())
        html.append(
            """<video alt="{}" autoplay
                    loop controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{}" type="video/mp4" />
                </video>""".format(
                mp4, video_b64.decode("ascii")
            )
        )
    ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [114]:
print(f"FQI agent on {env_id} after {n_iterations} iterations:")
show_videos("./logs/videos/", prefix=video_name)

FQI agent on CartPole-v1 after 20 iterations:


### Going further (optional)

- play with different models, and with their hyperparameters
- play with the discount factor
- play with the number of data collected/used
- combine data from random policy with data from trained model
- Use a neural network as the regression model (Scikit-learn has a class for simple fully connected neural networks)
- Implement DQN

## Conclusion

What we have seen in this notebook:
- collecting data using a random agent in a gym environment
- predicting q-values using a regression model
- the fitted q-iteration (FQI) algorithm to learn from an offline dataset